In [2]:
%run ../yp_utils.py

# Initial setup

In [3]:
paper_pmid = 34568930
paper_name = 'garcia_riekhof_2021' 

In [4]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [5]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [6]:
original_data = pd.read_excel('raw_data/Table1.xlsx',sheet_name='Sheet1')

In [7]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 58 x 4


In [8]:
original_data.head()

,Gene ID,Name,Previous ID,Description
0,YAL009W,SPO7,Yes,Subunit of Nem1p-Spo7p phosphatase holoenzyme
1,YAL039C,CYC3,No,Cytochrome c heme lyase
2,YBR097W,VPS15,No,Protein kinase involved in vacuolar protein so...
3,YBR184W,NaN,No,Putative protein of unknown function
4,YCR009C,RVS161,No,Amphiphysin-like lipid raft protein


In [9]:
original_data['orf'] = original_data['Gene ID'].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Gene ID, Name, Previous ID, Description, orf]
Index: []


In [13]:
original_data['data'] = 1

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['data']].copy()

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(58, 1)

# Prepare the final dataset

In [18]:
data = original_data.copy()

In [19]:
dataset_ids = [22054]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,22054
data_type,value
orf,
YAL009W,1
YAL039C,1
YBR097W,1
YBR184W,1
YCR009C,1


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22054
,data_type,value
gene_id,orf,
7,YAL009W,1
37,YAL039C,1
292,YBR097W,1
378,YBR184W,1
565,YCR009C,1


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22054          
data_type       value    valuez
gene_id orf                    
7       YAL009W     1  8.866831
37      YAL039C     1  8.866831
292     YBR097W     1  8.866831
378     YBR184W     1  8.866831
565     YCR009C     1  8.866831

# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 34568930...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.01it/s]

Updating the data_modified_on field...
